Небольшая вводная: проект реализован для небольшого ознакомления со сбором данных и обучения модели. Вы наверное скажете, что один из первых проектов на предсказания доллара - глупо. Но какой бы бредовой ни была идея предугадывания доллара, признайтесь, вы думали об этом)

На сбор данных ушло 30 минут 28 секунд. Вышла 321 запись.
Для обучения модели было решено использовать 257 записей, а для тестовой выделить 64 записи, причем тестовая запись - каждая 5 подряд из собранных данных. Второй раз выберем последние 64 записи и далее проведем небольшой анализ. Начнем.

In [126]:
import pandas

data = pandas.read_csv('dollar_prediction.csv')
data

,politics,economy,technology,oil,date,dollar
0,Персидский протест: как в Иране начались массо...,Всюду шеринг: что такое экономика совместного ...,Ускоритель Gyrfalcon Technology Laceli — конку...,Ежедневная добыча нефти в России выросла до 30...,2018-01-04,57.133195
1,Как России победить коррупцию\nРоссийские воен...,"Ничего личного, только бизнес. ""Альфа-банк"" от...",В процессорах Intel найдена критическая уязвим...,"5 проверенных масел для губ, которые спасут ва...",2018-01-07,57.096222
2,Эксперты назвали сроки окупаемости российской ...,"Вкладчики ""Центральной сберкассы"" в Крыму оста...",YI Technology и Google представили камеру YI H...,Гели-масла для душа: 7 лучших\nBHGE: за 2017 г...,2018-01-10,57.036775
3,Как борьба с иноагентами ударит по пользовател...,Доля теневой экономики в ВВП Узбекистана соста...,"«Просвещение» потребовало 3,7 млрд рублей за о...",Moldovan businessman sent down the power of Ka...,2018-01-13,56.609871
4,ЧВК в законе: зачем Сергей Лавров предлагает л...,Взыскать с России деньги французам мешает согл...,В технологии Intel Active Management Technolog...,"Прежде, чем купить: что нужно знать о масле Ma...",2018-01-16,56.439166
...,...,...,...,...,...,...
316,NYT узнала о предупреждении Лаврову от Госдепа...,На заводе под Калугой запустили производство п...,Сбербанк разработал технологию определения ......,По морям по волнам: выбираем правильное масло ...,2020-08-09,73.601845
317,В МВД Белоруссии сообщили о 3 тыс. задержанных...,На 7% сократилось число занятых в малом бизнес...,WSJ: TikTok тайно собирал данные Android-устро...,Норвегия сокращает объемы поставок нефти на 26...,2020-08-12,73.316880
318,Министры стран ЕС договорились ввести санкции ...,На 7% сократилось число занятых в малом бизнес...,WSJ: TikTok тайно собирал данные Android-устро...,Shell бросила вызов США созданием СП с “Газпро...,2020-08-15,73.296368
319,Мишустин встретился с Дегтяревым на фоне проте...,Санаторий для Лукашенко: об активах Белоруссии...,Apple проиграла патентный спор с Optis Wireles...,Oil Price: Япония может стать главным рынком д...,2020-08-18,73.117000


In [127]:
# test = data[:][5::5]
test = data[:][257:]
index = test.index
test

,politics,economy,technology,oil,date,dollar
257,Кремль опроверг сообщения об идее Путина объед...,Гаджи Султанов: «Республиканская инвестиционна...,Суд оштрафовал Twitter и Facebook за отказ пер...,"""Роснефть"" перевела на ""Восток Ойл"" Западно-Ир...",2020-02-14,63.559860
258,Контракт на доставку американцев на МКС может ...,Отечественные продукты полностью заменили импо...,«Яндекс» впервые раскрыл инвестиции в беспилот...,"""Роснефть"" перевела на ""Восток Ойл"" Западно-Ир...",2020-02-17,63.456760
259,Главы крупнейших музеев написали Путину о проб...,В Петербурге завод Hyundai выпустил двухмиллио...,Information and Communication Technology (ICT)...,Нефть. Цены по сортам на 19 февраля 2020\n«Газ...,2020-02-20,63.768304
260,Россия предложила Белоруссии снижать премию в ...,К криптовалютному проекту Facebook присоединил...,Information and Communication Technology (ICT)...,Vitol предсказывает восстановление цен на нефт...,2020-02-23,64.390334
261,Сурков объяснил свой уход из Кремля\nСША накаж...,Минздрав Грузии: приостанавливать сообщение с ...,В Kia научили коробку передач предсказывать бу...,ОПЕК и Ливия поддержали цены на нефть\nПластик...,2020-02-26,65.505103
...,...,...,...,...,...,...
316,NYT узнала о предупреждении Лаврову от Госдепа...,На заводе под Калугой запустили производство п...,Сбербанк разработал технологию определения ......,По морям по волнам: выбираем правильное масло ...,2020-08-09,73.601845
317,В МВД Белоруссии сообщили о 3 тыс. задержанных...,На 7% сократилось число занятых в малом бизнес...,WSJ: TikTok тайно собирал данные Android-устро...,Норвегия сокращает объемы поставок нефти на 26...,2020-08-12,73.316880
318,Министры стран ЕС договорились ввести санкции ...,На 7% сократилось число занятых в малом бизнес...,WSJ: TikTok тайно собирал данные Android-устро...,Shell бросила вызов США созданием СП с “Газпро...,2020-08-15,73.296368
319,Мишустин встретился с Дегтяревым на фоне проте...,Санаторий для Лукашенко: об активах Белоруссии...,Apple проиграла патентный спор с Optis Wireles...,Oil Price: Япония может стать главным рынком д...,2020-08-18,73.117000


In [128]:
train = data
train.drop(test.index, inplace=True)
train

,politics,economy,technology,oil,date,dollar
0,Персидский протест: как в Иране начались массо...,Всюду шеринг: что такое экономика совместного ...,Ускоритель Gyrfalcon Technology Laceli — конку...,Ежедневная добыча нефти в России выросла до 30...,2018-01-04,57.133195
1,Как России победить коррупцию\nРоссийские воен...,"Ничего личного, только бизнес. ""Альфа-банк"" от...",В процессорах Intel найдена критическая уязвим...,"5 проверенных масел для губ, которые спасут ва...",2018-01-07,57.096222
2,Эксперты назвали сроки окупаемости российской ...,"Вкладчики ""Центральной сберкассы"" в Крыму оста...",YI Technology и Google представили камеру YI H...,Гели-масла для душа: 7 лучших\nBHGE: за 2017 г...,2018-01-10,57.036775
3,Как борьба с иноагентами ударит по пользовател...,Доля теневой экономики в ВВП Узбекистана соста...,"«Просвещение» потребовало 3,7 млрд рублей за о...",Moldovan businessman sent down the power of Ka...,2018-01-13,56.609871
4,ЧВК в законе: зачем Сергей Лавров предлагает л...,Взыскать с России деньги французам мешает согл...,В технологии Intel Active Management Technolog...,"Прежде, чем купить: что нужно знать о масле Ma...",2018-01-16,56.439166
...,...,...,...,...,...,...
252,Замглавы Минздрава Елену Бойко уволили после в...,России пророчат испытание низкими ценами на не...,Книжные издательства теряют больше 55 млрд руб...,"Запасы нефти в США за неделю увеличились на 0,...",2020-01-30,63.171548
253,Патриарх предложил вписать веру в Бога в Конст...,Мария Львова-Белова поборется за звание победи...,Книжные издательства теряют больше 55 млрд руб...,«Дочке» Объединенной судостроительной корпорац...,2020-02-02,63.642327
254,Завод УЗГА получил заказы на военные беспилотн...,Конференция Blockchain Economy в Стамбуле осве...,«Яндекс» нашел новый канал сбыта бортовых ...\...,"""Роснефть"" подписала контракт с Indian Oil на ...",2020-02-05,62.625420
255,Счетная палата назвала главные проблемы россий...,В Ярославской области строительство просело из...,IT-технологии на службе маркетинга. Старт рабо...,"""Роснефть"" подписала контракт с Indian Oil на ...",2020-02-08,63.834716


In [129]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

Далее необходимо отредактировать данные.
Преобразую все буквы к нижнему регистру и заменю все символы кроме букв и цифр на пробел. Также нужно было что-то сделать с датой. Решено было из даты выделить год и квартал.

In [130]:
def transform_data(train):
  quart = []
  years = []

  for i in range(len(train['dollar'])):
    # приведем все буквы в нижний регистр
    train['politics'][i] = train['politics'][i].lower()
    train['economy'][i] = train['economy'][i].lower()
    train['technology'][i] = train['technology'][i].lower()
    train['oil'][i] = train['oil'][i].lower()
  
    # также преобразуем 'date' в соответствующий квартал
    month, year = train['date'][i].split('-')[1], train['date'][i].split('-')[0]
    quart.append(int(month) // 4 + 1)
    years.append(int(year))

  del train['date']
  train['year'] = years
  train['quart'] = quart

  # заменим все кроме цифр и букв на пробел
  for column in train.columns:
    train[column] = train[column].replace('[^a-zA-Zа-яА-Я0-9]', ' ', regex = True)
  return train

train = transform_data(train)
test = transform_data(test)

train

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: Setti

,politics,economy,technology,oil,dollar,year,quart
0,персидский протест как в иране начались массо...,всюду шеринг что такое экономика совместного ...,ускоритель gyrfalcon technology laceli конку...,ежедневная добыча нефти в россии выросла до 30...,57.133195,2018,1
1,как россии победить коррупцию российские военн...,ничего личного только бизнес альфа банк от...,в процессорах intel найдена критическая уязвим...,5 проверенных масел для губ которые спасут ва...,57.096222,2018,1
2,эксперты назвали сроки окупаемости российской ...,вкладчики центральной сберкассы в крыму оста...,yi technology и google представили камеру yi h...,гели масла для душа 7 лучших bhge за 2017 г ...,57.036775,2018,1
3,как борьба с иноагентами ударит по пользовател...,доля теневой экономики в ввп узбекистана соста...,просвещение потребовало 3 7 млрд рублей за о...,moldovan businessman sent down the power of ka...,56.609871,2018,1
4,чвк в законе зачем сергей лавров предлагает л...,взыскать с россии деньги французам мешает согл...,в технологии intel active management technolog...,прежде чем купить что нужно знать о масле ma...,56.439166,2018,1
...,...,...,...,...,...,...,...
252,замглавы минздрава елену бойко уволили после в...,россии пророчат испытание низкими ценами на не...,книжные издательства теряют больше 55 млрд руб...,запасы нефти в сша за неделю увеличились на 0 ...,63.171548,2020,1
253,патриарх предложил вписать веру в бога в конст...,мария львова белова поборется за звание победи...,книжные издательства теряют больше 55 млрд руб...,дочке объединенной судостроительной корпорац...,63.642327,2020,1
254,завод узга получил заказы на военные беспилотн...,конференция blockchain economy в стамбуле осве...,яндекс нашел новый канал сбыта бортовых ...,роснефть подписала контракт с indian oil на ...,62.625420,2020,1
255,счетная палата назвала главные проблемы россий...,в ярославской области строительство просело из...,it технологии на службе маркетинга старт рабо...,роснефть подписала контракт с indian oil на ...,63.834716,2020,1


Выделим все 4 столбца текста в один для дальнейшего представления его в векторы признаков

In [131]:
def get_text_data(train):
  train_ = []
  for i in range(len(train['dollar'])):
    train_.append(train['politics'][i] + ' ' + train['economy'][i] + ' ' + train['technology'][i] + ' ' + train['oil'][i])
  return train_
train_ = get_text_data(train)
test_ = get_text_data(test)

Оставлю только те слова, которые встречаются хотя бы в 5 объектах

In [132]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.min_df = 5
tags = ['politics', 'economy', 'technology', 'oil']
X_train = vectorizer.fit_transform(train_)
X_test = vectorizer.transform(test_)

Для столбцов 'year' и 'quart' применю one-hot кодирование

In [133]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(train[['year', 'quart']].to_dict('records'))
X_test_categ = enc.transform(test[['year', 'quart']].to_dict('records'))

Объединю полученные признаки в матрицу объектов-признаков

In [134]:
from scipy.sparse import hstack
train_matrix = hstack([X_train, X_train_categ])
test_matrix = hstack([X_test, X_test_categ])

Ну и наконец - обучение гребневой регрессии

In [135]:
from sklearn.linear_model import Ridge
y_train = train['dollar']
ridge = Ridge(alpha = 1, random_state = 241)
ridge.fit(train_matrix, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=241, solver='auto', tol=0.001)

Финал. Получение прогнозов

In [138]:
answer = ridge.predict(test_matrix)
for i in range(len(answer)):
  print('{} - {}'.format(i, answer[i]))

0 - 62.985773887558935
1 - 62.968524294391045
2 - 64.03370407578996
3 - 63.37665534624921
4 - 63.29751076195589
5 - 63.4633594316183
6 - 63.981295664572826
7 - 64.66923289061924
8 - 63.37563808860068
9 - 62.74058467547229
10 - 63.19330425157477
11 - 63.467095543262076
12 - 64.06474889512538
13 - 63.440173979978226
14 - 62.906048417558395
15 - 63.887886261733456
16 - 63.95859381041191
17 - 64.71846654889396
18 - 65.49294621631407
19 - 65.4365193105923
20 - 64.96258790745742
21 - 64.11350151144961
22 - 65.79501200902087
23 - 65.63064356825089
24 - 65.20180884071942
25 - 64.67115292607332
26 - 65.12796511493934
27 - 64.22779404524408
28 - 65.12143885273645
29 - 65.62681290223964
30 - 65.3748236757956
31 - 64.50595939139976
32 - 64.59753914900602
33 - 65.48719285552261
34 - 64.27054883924393
35 - 64.59461922842593
36 - 63.671111056912196
37 - 64.59419105789902
38 - 65.49505241145289
39 - 65.1097931143463
40 - 64.86374552831285
41 - 64.80526236127162
42 - 64.63866792486169
43 - 64.766776945

Создадим таблицу, для дальнейшего удобного просмотра результатов

In [122]:
data = pandas.read_csv('dollar_prediction.csv')
real_values = data.loc[data.index.isin(index)][['dollar', 'date']]
real_values = real_values.reset_index(drop=True)
real_values

,dollar,date
0,63.559860,2020-02-14
1,63.456760,2020-02-17
2,63.768304,2020-02-20
3,64.390334,2020-02-23
4,65.505103,2020-02-26
...,...,...
59,73.601845,2020-08-09
60,73.316880,2020-08-12
61,73.296368,2020-08-15
62,73.117000,2020-08-18


In [124]:
table = pandas.DataFrame({'date': real_values['date'], 'prediction': answer, 'real_value': real_values['dollar'], 'index': index}, index=None)
table.set_index('date', inplace=True)
#table.to_csv('latest_data.csv')
table

,prediction,real_value,index
date,,,
2020-02-14,62.985774,63.559860,257
2020-02-17,62.968524,63.456760,258
2020-02-20,64.033704,63.768304,259
2020-02-23,63.376655,64.390334,260
2020-02-26,63.297511,65.505103,261
...,...,...,...
2020-08-09,66.305235,73.601845,316
2020-08-12,66.962443,73.316880,317
2020-08-15,67.657167,73.296368,318
